# Classifying Tumors in Mammograms

# Part 1B - MIAS Base CNN Modeling

The [MIAS dataset](http://peipa.essex.ac.uk/info/mias.html) was the initial dataset used to explore classifying images for tumor identification in mammograms. 

## Contents
- [Data Import](#Data-Import-&-Cleaning)
- [Prepare Images for Modeling](#Prepare-images-for-modeling)
- [Modeling](#Modeling)
    - [CNN](#CNN)
    - [Other Examples of CNN Attempts](#Other-Examples-of-CNN-Attempts)
    - [Bayes Search](#Bayes-Search-to-try-and-Optimize-the-CNN)
- [Conclusions](#Conclusions)

## Overview

### Modeling Techniques Explored

- Resizing
- Gray scaling of image
- Reflection - flip images horizontally and vertically
- Rotation - Form of upsampling your dataset since it creates rotated copies of the same image.


### Choice of Hyperparameters
- Loss function - binary_crossentropy
- Optimizer - Adam (Looking for a steady decrease in loss rate)
- Metrics - Since we're dealing with life and death cancer diagnoses, False Negatives are far more detrimental since they mean you miss diiagnosing a tumor so reducing False Negatives is the aim. Also should pay attention to Accuracy and F1 score. Accuracy is important since we care how many True Positives our model finds but the F1 score is the harmonic mean of the model's Precision and Recall and is used when the False Negatives and False Positives might be more critical. In our classification problem, there is definitely imbalanced class distribution and thus F1-score might be a better metric to evaluate our model on.
- Learning Rate / Epochs - Aimed for slower learning rates
- The sigmoid function is used for the two-class logistic regression, whereas the softmax function is used for the multiclass logistic regression. In our case, we used the sigmoid for the tumor/no tumor classification. If we were to expand this project to tumor types we would need to use the softmax function.

In [1]:
#pip install tensorflow

In [2]:
#pip install --upgrade category_encoders

In [3]:
#pip install scikit-optimize

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l1, l2, l1_l2
import tensorflow as tf
import glob
import category_encoders as ce
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras import utils

from skopt.space import Integer, Real, Categorical
from skopt import BayesSearchCV

AttributeError: module 'tensorflow.core.framework.types_pb2' has no attribute 'SerializedDType'

## Data Import

In [ ]:
#Files located in ./all-mias.tar/
#images all start with mdb

files = glob.glob('mias_data/all-mias/mdb*')
#print(files)

In [ ]:
#Read in the files
#image_size sets the image dimensions, base is 1024x1024
image_size = 128
images = []
#print("reached here")
for file in files:
    #print("reached here 2")
    image = cv2.imread(file,0)
    image = cv2.resize(image, (image_size, image_size))
    images.append(image)
    

In [ ]:
#Display First ten images
print (len(images))
for i in range(10):
    plt.imshow(images[i],cmap=plt.get_cmap('gray'))
    plt.show()

## Load in the documentation on tumors

Text file provided with:
MIAS database reference number.

 - Character of background tissue:

        F  Fatty
        G  Fatty-glandular
        D  Dense-glandular
     

 - Class of abnormality present:

          CALC  Calcification
          CIRC  Well-defined/circumscribed masses
          SPIC  Spiculated masses
          MISC  Other, ill-defined masses
          ARCH  Architectural distortion
          ASYM  Asymmetry
          NORM  Normal

 - Severity of abnormality;

          B  Benign
          M  Malignant
      
 - x,y image-coordinates of centre of abnormality.

 - Approximate radius (in pixels) of a circle enclosing the abnormality. 

In [ ]:
#Read in CSV, cleaned separately of extraneous information
labels = pd.read_csv('mias_data/all-mias/info_red.csv',delimiter=',',names=['ID','bg','ab_type','diagnosis','ab_x','ab_y','ab_r'])

In [ ]:

#print(labels)

In [ ]:
#If no tumor, make an N
#Add tumor column
labels['diagnosis'].fillna('N',inplace=True)
labels['tumor'] = labels['diagnosis'].map(lambda x: 0 if x == 'N' else 1) # new list called tumor for 0 and 1's 
labels['tumor'].value_counts(normalize=True)

In [ ]:
#Make mask layout
labels['ab_x'].fillna(0,inplace=True)
labels['ab_y'].fillna(0,inplace=True)
labels['ab_r'].fillna(0,inplace=True)

In [ ]:
#Check for the biggest and smallest box
labels[labels['ab_r'] > 0]['ab_r'].max(),labels[labels['ab_r'] > 0]['ab_r'].max()

In [ ]:
#Check that all empty things are filled
labels.info()

In [ ]:
#330 entries, 322 images, some duplications
labels['ID'] = labels['ID'].map(lambda x: x[3:])
labels['ID'].value_counts()

In [ ]:
#Check breakdown of tumor diagnosis
labels['diagnosis'].value_counts(normalize=True)

## Prepare images for modeling

 - Scale the images
 - Setup X and y
    - Classify y two ways, either tumor or not, or type of tumor
    - Loss function, final activation, and output layer size modified

In [ ]:
print(images[1])
images = [image/255 for image in images]

In [ ]:
print(images)

In [ ]:
images = np.array(images)

In [ ]:
print(images)

Comment out one of the next two blocks, first block does classification on tumor type, next does it just on whether or not there is a tumor

In [ ]:
# X = np.expand_dims(images,axis=3)
# y = labels.drop_duplicates(subset='ID')[['ab_type']]
# encoder = ce.OneHotEncoder(cols=['ab_type'], use_cat_names=True,
#                         return_df=False)
# print(y['ab_type'].nunique())
# y = encoder.fit_transform(y)
# loss_func, act_func, out_layer = 'categorical_crossentropy', 'softmax', 7


In [ ]:
X = np.expand_dims(images,axis=3)
#print("x is this")
#print(X)
y = labels.drop_duplicates(subset='ID')['tumor']
loss_func, act_func, out_layer = 'binary_crossentropy', 'sigmoid', 1

#print(y)
y.shape
#X.shape

Train test split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42)

# "train_test_split" function to split your dataset
# Takes in: 
    ##: input data and corresponding labels
# Returns: 
    ##: multiple arrays or matrices representing the train-test splits
    
# X represents: input data
# Y respresents: corresponding labels
# random_state: ensures reproducibility by fixing the random seed

# function returns four arrays: X_train, X_test, y_train, and y_test
# data is split randomly, ensuring that both the training and testing subsets have a representative distribution of the original data

# what is the point of doimg this? And splitting the data randomly? 
# By splitting the data into separate training and testing sets, you can assess the generalization ability of your model 
        #and estimate its performance on unseen data, 
        #helping you make informed decisions about the model's effectiveness and potential areas of improvement.

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

## Modeling

Various sets of CNN fits were tried, with the following:
 - Number and size of hidden Convolution layers (1 to 4 layers, 10-128 filters)
 - Number and size Dense Layers (1-5 dense layers, 120-2000 nodes)
 - Various forms of regularization (L1,Batch, and Dropout)
 - Image size (100-1024 pixels)
 - Rotations


## CNN

In [ ]:
#Instantiate model
cnn = Sequential()

# The Sequential class is used to create a sequential model, 
# where layers are stacked one after another to form the neural network architecture


#convolution layers       -----------------------# defines the architecture of your neural network by adding layers in a sequential manner as shown below
# -----------------
# -----------------
cnn.add(Conv2D(filters = 20,       #number of filters to use
              kernel_size = (4,4),   #dimensions of the filters
              activation = 'relu',    #activation function
              input_shape = (image_size, image_size,1)))  #shape of image
cnn.add(MaxPooling2D(pool_size = (2,2))) 
cnn.add(Conv2D(20, kernel_size = (4,4), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2,2)))
cnn.add(Flatten())

#dense layers
# saw an example of this on Chat GPT 
# -------------
# We are using 2 dense layers 
cnn.add(Dense(128, activation = 'relu'))   # the first layer has 128 units 
cnn.add(Dense(48, activation = 'relu'))    # the second layer has 48 units 
# The term "units" refers to the number of neurons in a particular layer of the network
# Each unit or neuron represents a learnable parameter that processes input data and produces an output 
# The number of units in a layer has an impact on the complexity and capacity of the neural network. 
#Increasing the number of units generally increases the model's ability to capture complex patterns and representations in the data.


# Example lines of the regularization methods tried
# cnn.add(Dropout(0.2))
# cnn.add(BatchNormalization())
# ,kernel_regularizer=l2(0.01)
# cnn.add(Dropout(0.3))

#finalize things
cnn.add(Dense(out_layer, activation = act_func))#  the Dense class represents a fully connected layer where each neuron is connected to every neuron in the previous layer.
                                                # The out_layer parameter specifies the number of units (neurons) in this dense layer,
                                                # act_func specifies the activation function to be used. The activation function adds non-linearity to the output of the neurons.
    cnn.compile(loss = loss_func,               # The loss function measures the discrepancy between the predicted outputs and the true values.# 
           optimizer = 'adam',                  # The adam optimizer is a popular optimization algorithm used to update the weights of the neural network during the training process.
           metrics = ['accuracy'])              # 'accuracy' metric is used, which calculates the accuracy of the model's predictions compared to the true labels.
    
    
    

In [ ]:
# Early stop to end model, patience varied as needed when changin models
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')


# validation loss (val_loss): The training process will monitor the validation loss and make decisions based on its behavior.
# min_delta=0: This parameter defines the minimum change in the monitored metric that qualifies as an improvement. 
            # A value of 0 means any decrease in the metric will be considered an improvement. 
            # If the change in the monitored metric is smaller than min_delta, it will not be considered as an improvement.
# The patience parameter determines the number of epochs to wait before stopping the training if no improvement is observed.
# In this case, verbose=1 means the callback will provide detailed output.
# 'auto' mode automatically determines the direction of improvement based on the metric being monitored. 
            # For example, if the monitored metric is loss, it will assume a lower value is better.

    
# By using the EarlyStopping callback, you can prevent overfitting and save computational resources by 
        # stopping the training process early when the monitored metric (in this case, validation loss) is no longer improving
    # This helps in avoiding unnecessary training iterations and allows for efficient model training.




In [ ]:
#Perform the fit, epochs set to 1000 since early stop being used. # Not getting any better so stop before a 1000 iterations 
tf.random.set_seed(42)
history_bw = cnn.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 16,
                 epochs = 1000,
                 verbose = 1,
                 callbacks = [early_stop]
)

# The cnn.fit method executes the training process, 
# where the model's weights are updated using the provided training data, loss function, and optimization algorithm. 
# The model's performance is monitored on the validation data to prevent overfitting 
# and determine when to stop training early if necessary.

In [ ]:
#Make predictions, round since output is in probability
preds = cnn.predict(X_test).round(0)
#Base score
metrics.f1_score(y_test,preds),metrics.accuracy_score(y_test,preds)

In [ ]:
#Confusion matrix
cm = confusion_matrix(y_test, preds)

In [ ]:
#plot CM
ConfusionMatrixDisplay(cm).plot()

In [ ]:
#Function for plotting CNN history
def Loss_Acc_Plot(acc,val_acc,loss,val_loss):
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12,8))
    fig.suptitle("Model Performance", fontsize = 25, fontweight = 'bold')
    fig.savefig('Accuracy_Loss_figure.png')

    ax1.plot(range(1, len(acc) + 1), acc,  color='#185fad')
    ax1.plot(range(1, len(val_acc) + 1), val_acc, color='orange')
    ax1.set_title('History of Accuracy by Epoch', fontsize=20)
    ax1.set_xlabel('Epoch', fontsize=18)
    ax1.set_ylabel('Accuracy', fontsize=18)
    ax1.legend(['training', 'validation'], fontsize=18)


    ax2.plot(range(1, len(loss) + 1), loss, color='#185fad')
    ax2.plot(range(1, len(val_loss) + 1), val_loss, color='orange')
    ax2.set_title('History of Loss by Epoch', fontsize=20)
    ax2.set_xlabel('Epoch', fontsize=18)
    ax2.set_ylabel('Binary Crossentropy (Loss)', fontsize=18)
    ax2.legend(['training', 'validation'], fontsize=18)
    
    plt.tight_layout()
    plt.show()
    

# Understand this graphs 

In [ ]:
#run the plot for the history
Loss_Acc_Plot(history_bw.history['accuracy'],history_bw.history['val_accuracy'],
               history_bw.history['loss'],history_bw.history['val_loss'])

## Other Examples of CNN Attempts

The following shows some of the other example nets tried, rows were deactivated or modified to try different things. 

In [ ]:
# tf.random.set_seed(42)
# cnn = Sequential()
# pool_size = 2
# cnn.add(Conv2D(filters = 32,       #number of filters to use
#               kernel_size = (4,4),   #dimensions of the filters
#               activation = 'relu',    #activation function
#               input_shape = (image_size,image_size,1), #shape of image
#               padding='valid', 
#               #strides=(pool_size,pool_size)
#               ))  
# cnn.add(MaxPooling2D(pool_size = (pool_size,pool_size))) 
# cnn.add(Conv2D(32, kernel_size = (4,4), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size = (pool_size,pool_size)))
# # cnn.add(Conv2D(20, kernel_size = (4,4), activation = 'relu'))
# # cnn.add(MaxPooling2D(pool_size = (pool_size,pool_size)))
# cnn.add(Flatten())
# # cnn.add(Dense(32, activation = 'relu'))
# # cnn.add(Dense(32, activation = 'relu'))
# # cnn.add(Dense(32, activation = 'relu'))
# # cnn.add(Dense(32, activation = 'relu'))
# # cnn.add(Dense(32, activation = 'relu'))
# cnn.add(Dropout(0))
# cnn.add(Dense(1024, activation = 'relu'))
# cnn.add(Dropout(0.2))
# # cnn.add(Dense(32, activation = 'relu'))

# cnn.summary()
# cnn.add(Dense(out_layer, activation = act_func))
# cnn.compile(loss = loss_func,
#            optimizer = 'adam',
#            metrics = ['accuracy'])
# # history_bw = cnn.fit(X_train, y_train, validation_data = (X_test, y_test),
# #                  batch_size = 32,
# #                  epochs = 1000,
# #                  verbose = 1,
# #                  callbacks = [early_stop]
# # )

In [ ]:
# cnn = Sequential()
# cnn.add(Conv2D(filters = 20,       #number of filters to use
#               kernel_size = (4,4),   #dimensions of the filters
#               activation = 'relu',    #activation function
#               input_shape = (image_size,image_size,1)))  #shape of image
# cnn.add(MaxPooling2D(pool_size = (2, 2))) 
# cnn.add(Conv2D(20, kernel_size = (4,4), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size = (2, 2)))
# cnn.add(Flatten())
# cnn.add(Dense(2048, activation = 'relu'))
# #cnn.add(Dense(512, activation = 'relu'))
# #cnn.add(Dense(32, activation = 'relu'))
# cnn.add(Dense(1, activation = 'sigmoid'))
# cnn.compile(loss = 'binary_crossentropy',
#            optimizer = 'adam',
#            metrics = ['accuracy'])

In [ ]:
# cnn = Sequential()

# cnn.add(Conv2D(32, (3,3), padding='valid', strides=(1, 1),input_shape = (image_size,image_size,1), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(3, 3)))

# cnn.add(Conv2D(64, (3,3), padding='same', strides=(1, 1), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(3, 3),strides=2))
# # cnn.add(Dropout(0.2))

# # cnn.add(Conv2D(64, (5,5), padding='same', strides=(1, 1), activation = 'relu'))
# # cnn.add(MaxPooling2D(pool_size=(3, 3),strides=2))

# # cnn.add(Conv2D(32, (5,5), padding='same', strides=(1, 1), activation = 'relu'))
# # cnn.add(MaxPooling2D(pool_size=(2, 2),strides=2))

# # cnn.add(Dropout(0.2))
# cnn.add(Flatten())
# cnn.add(Dense(128, activation = 'relu'))
# cnn.add(Dense(64, activation = 'relu'))
# cnn.add(Dense(out_layer, activation = act_func))
# cnn.compile(loss = loss_func,
#            optimizer = 'adam',
#            metrics = ['accuracy'])

In [ ]:
# cnn = Sequential()

# cnn.add(Conv2D(filters = 32,       #number of filters to use
#               kernel_size = (3, 3),   #dimensions of the filters
#               activation = 'relu',    #activation function
#               input_shape = (128,128,1), #shape of image
#               #padding='valid', 
#               #strides=(1, 1)
#               ))  
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# #cnn.add(Dropout(0.25))
# cnn.add(Flatten())
# cnn.add(Dense(96, activation = 'relu'))
# cnn.add(Dense(1, activation = 'sigmoid'))
# cnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# early_stop = EarlyStopping(monitor = 'val_loss', 
#                            min_delta = 0,
#                            patience = 5, 
#                            verbose = 1, 
#                            mode='auto', 
#                            restore_best_weights=True)
# # Fit model on training data
# history_bw = cnn.fit(X_train, y_train, validation_data = (X_test, y_test),
#                  batch_size = 64,
#                  epochs = 100,
#                  verbose = 1,
#                  callbacks=[early_stop]
# )

In [ ]:
# cnn = Sequential()

# cnn.add(Conv2D(filters = 64,       #number of filters to use
#               kernel_size = (3, 3),   #dimensions of the filters
#               activation = 'relu',    #activation function
#               input_shape = (128,128,1), #shape of image
#               #padding='valid', 
#               #strides=(1, 1)
#               ))  
# cnn.add(MaxPooling2D(pool_size=(2, 2)))

# cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# cnn.add(Dropout(0.25))
# cnn.add(Flatten())
# cnn.add(Dense(96, activation = 'relu'))
# cnn.add(Dense(64, activation = 'relu'))
# cnn.add(Dense(64, activation = 'relu'))
# cnn.add(Dense(1, activation = 'sigmoid'))
# cnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# early_stop = EarlyStopping(monitor = 'val_loss', 
#                            min_delta = 0,
#                            patience = 5, 
#                            verbose = 1, 
#                            mode='auto', 
#                            restore_best_weights=True)
# # Fit model on training data
# history_bw = cnn.fit(X_train, y_train, validation_data = (X_test, y_test),
#                  batch_size = 64,
#                  epochs = 100,
#                  verbose = 1,
#                  callbacks=[early_stop]
# )

## Bayes Search to try and Optimize the CNN

Attempted a Bayes search with several levels of varying many of the parameters. The results did not turn out well

In [ ]:
#function to create a model for the Bayes search

# def model_func(first_fil, sec_fil, kernel_1,kernel_2,pool_1,pool_2,layer_1,layer_2,layer_3,dropout):
#     cnn = Sequential()
#     cnn.add(Conv2D(filters = first_fil,       #number of filters to use
#                   kernel_size = (kernel_1,kernel_1),   #dimensions of the filters
#                   activation = 'relu',    #activation function
#                   input_shape = (image_size, image_size,1)))  #shape of image
#     cnn.add(MaxPooling2D(pool_size = (pool_1, pool_1))) 
#     cnn.add(Conv2D(sec_fil, kernel_size = (kernel_2, kernel_2), activation = 'relu'))
#     cnn.add(MaxPooling2D(pool_size = (pool_2, pool_2)))
#     cnn.add(Flatten())
#     cnn.add(Dense(layer_1, activation = 'relu'))
#     cnn.add(Dropout(dropout))
#     cnn.add(Dense(layer_2, activation = 'relu'))
#     cnn.add(Dropout(dropout))
#     cnn.add(Dense(layer_3, activation = 'relu'))
#     cnn.add(Dropout(dropout))
#     cnn.add(Dense(1, activation = 'sigmoid'))
#     cnn.compile(loss = 'binary_crossentropy',
#                optimizer = 'adam',
#                metrics = ['accuracy'])
#     return cnn

In [ ]:
#Create classifier pipeline
# nn = KerasClassifier(build_fn = model_func,batch_size = 32, epochs = 12,verbose = 1)

In [ ]:
# Bayes Search parameter grid

# params = {'first_fil':Integer(4,64),
#          'sec_fil':Integer(4,64),
#          'kernel_1':Integer(2,10),
#          'kernel_2':Integer(2,10),
#          'pool_1':Integer(2,5),
#          'pool_2':Integer(2,5),
#          'layer_1':Integer(10,64),
#          'layer_2':Integer(10,64),
#          'layer_3':Integer(10,64),
#          'dropout':Real(0.01,0.3)}

In [ ]:
#Run the Bayes search

# cnn_bs = BayesSearchCV(estimator = nn,
#                      search_spaces = params,
#                      n_iter = 200,
#                      cv = 3,
#                      verbose = 1,
#                      random_state=42)
# cnn_bs.fit(X_train,y_train)

## Conclusions

- Massively overfit models and performance is not much better than baseline 62.7%
- Need more training data and to explore more image preprocessing and data augmentation methods
- Be sure to Train test split before augmenting any images. When you rotate images it creates copies and if you split these after the fact you will most likely cause data leakage